In [ ]:
import numpy as np
import pandas as pd
from datasets import Dataset
from gluonts.dataset.split import split
from matplotlib import pyplot as plt

from utils.prerocess import read_stock_df


# 資料準備
將資料讀取並抽取2010到2011的資料

In [ ]:
df = read_stock_df('dataset/index/GSPC.csv', start_date='2010-01-01', end_date='2011-01-01',
                   drops_columns=['Dividends', 'Stock Splits'])

df


In [ ]:
series_target = df['Close']

dataset_std = np.nanstd(series_target)
dataset_mean = np.nanmean(series_target)

dataset_mean


In [ ]:
from gluonts.dataset.pandas import PandasDataset

freq = "1D"
dataset = PandasDataset(df, target="Close", freq=freq, timestamp='Date')

dataset


In [ ]:
from gluonts.dataset.util import to_pandas


def highlight_entry(entry, color):
    start = entry["start"]
    end = entry["start"] + len(entry["target"])
    plt.axvspan(start, end, facecolor=color, alpha=0.2)


def plot_dataset_splitting(original_dataset, training_dataset, test_pairs):
    for original_entry, train_entry in zip(original_dataset, training_dataset):
        to_pandas(original_entry).plot()
        highlight_entry(train_entry, "red")
        plt.legend(["sub dataset", "training dataset"], loc="upper left")
        plt.show()

    for original_entry in original_dataset:
        for test_input, test_label in test_pairs:
            to_pandas(original_entry).plot()
            highlight_entry(test_input, "green")
            highlight_entry(test_label, "blue")
            plt.legend(["sub dataset", "test input", "test label"], loc="upper left")
            plt.show()
            break


In [ ]:
df['Date'].iloc[-72]


In [ ]:
length = len(df)
split_size = length // 10 * 2
split_size = len(df['Date'].iloc[-split_size:])
split_date = df['Date'].iloc[-split_size]

# 要弄圖的時候記得改成5
prediction_length = 1
windows = split_size // prediction_length - 1

train_dataset, test_template = split(
    dataset, date=pd.Period(split_date, freq=freq)
)

test_pairs = test_template.generate_instances(
    prediction_length=prediction_length,
    windows=windows,
)

plot_dataset_splitting(dataset, train_dataset, test_pairs)


In [ ]:
# length = len(df)
# split_size = length // 10 * 2
#
# prediction_length = 1
#
# train_dict = {
#     'start': [df['Date'].iloc[0]],
#     'target': [df['Close'][:-split_size]],
# }
#
# test_dict = {
#     'start': [df['Date'].iloc[0] for i in range(split_size)],
#     'target': [df['Close'].iloc[:-split_size + i] for i in range(split_size)],
# }
#
# train_dict


In [ ]:
# train_dataset = Dataset.from_dict(train_dict)
# test_dataset = Dataset.from_dict(test_dict)

# assert len(train_dataset[0]['target']) + prediction_length == len(validation_dataset[0]['target'])

# figure, axes = plt.subplots()
# axes.plot(train_dataset[0]['target'], color="blue")
# axes.plot(validation_dataset[0]['target'], color="red", alpha=0.5)
#
# plt.show()


In [ ]:
from gluonts.time_feature import get_lags_for_frequency

lags_sequence = get_lags_for_frequency(freq)
lags_sequence = lags_sequence[:10]
lags_sequence


In [ ]:
from gluonts.time_feature import time_features_from_frequency_str

time_features = time_features_from_frequency_str(freq)

time_features


In [ ]:
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction

context_length = 9

config = TimeSeriesTransformerConfig(
    prediction_length=prediction_length,
    context_length=context_length,  # context length
    lags_sequence=lags_sequence,
    num_time_features=len(time_features) + 1,  # we'll add 2 time features ("month of year" and "age", see further)
    num_static_categorical_features=1,  # we have a single static categorical feature, namely time series ID
    cardinality=[len(train_dataset)],  # it has 366 possible values
    embedding_dimension=[2],  # the model will learn an embedding of size 2 for each of the 366 possible values
    encoder_layers=4,
    decoder_layers=4,
)

model = TimeSeriesTransformerForPrediction(config)


## Define Transformations

Next, we define the transformations for the data, in particular for the creation of the time features (based on the dataset or universal ones).

Again, we'll use the GluonTS library for this. We define a `Chain` of transformations (which is a bit comparable to `torchvision.transforms.Compose` for images). It allows us to combine several transformations into a single pipeline.

In [ ]:
from gluonts.time_feature import time_features_from_frequency_str
from gluonts.dataset.field_names import FieldName
from gluonts.transform import (
    AddAgeFeature,
    AddObservedValuesIndicator,
    AddTimeFeatures,
    AsNumpyArray,
    Chain,
    ExpectedNumInstanceSampler,
    InstanceSplitter,
    SelectFields,
    SetField,
    TestSplitSampler,
    Transformation,
    ValidationSplitSampler,
    VstackFeatures,
    RenameFields,
)


In [ ]:
from transformers import PretrainedConfig


def create_transformation(freq: str, config: PretrainedConfig) -> Transformation:
    # a bit like torchvision.transforms.Compose
    return Chain(
        [
            # step 1: remove static/dynamic fields if not specified
            # RemoveFields(field_names=['Dividends', 'Stock Splits']),

            # step 2: use static features if available, if not add dummy values
            SetField(output_field=FieldName.FEAT_STATIC_CAT, value=[0]),

            SetField(output_field=FieldName.FEAT_STATIC_REAL, value=[0.0]),

            # step 3: convert the data to NumPy (potentially not needed)
            AsNumpyArray(
                field=FieldName.FEAT_STATIC_CAT,
                expected_ndim=1,
                dtype=int,
            ),
            AsNumpyArray(
                field=FieldName.FEAT_STATIC_REAL,
                expected_ndim=1,
            ),
            # AsNumpyArray(
            #     field='open',
            #     # in the following line, we add 1 for the time dimension
            #     expected_ndim=1,
            # ),
            AsNumpyArray(
                field=FieldName.TARGET,
                # in the following line, we add 1 for the time dimension
                expected_ndim=1,
            ),
            # step 4: handle the NaN's by filling in the target with zero
            # and return the mask (which is in the observed values)
            # true for observed values, false for nan's
            # the decoder uses this mask (no loss is incurred for unobserved values)
            # see loss_weights inside the xxxForPrediction model
            AddObservedValuesIndicator(
                target_field=FieldName.TARGET,
                output_field=FieldName.OBSERVED_VALUES,
            ),

            # step 5: add temporal features based on freq of the dataset
            # month of year in this case
            # these serve as positional encodings
            # pure time feature, not target info inside
            AddTimeFeatures(
                start_field=FieldName.START,
                target_field=FieldName.TARGET,
                output_field=FieldName.FEAT_TIME,
                time_features=time_features_from_frequency_str(freq),
                pred_length=config.prediction_length,
            ),

            # step 6: add another temporal feature (just a single number)
            # tells the model where in the life the value of the time series is
            # sort of running counter
            AddAgeFeature(
                target_field=FieldName.TARGET,
                output_field=FieldName.FEAT_AGE,
                pred_length=config.prediction_length,
                log_scale=True,
            ),

            # step 7: vertically stack all the temporal features
            VstackFeatures(
                output_field=FieldName.FEAT_TIME,
                input_fields=[FieldName.FEAT_TIME, FieldName.FEAT_AGE],
            ),

            # step 8: rename to match HuggingFace names

            RenameFields(
                mapping={
                    FieldName.FEAT_STATIC_CAT: "static_categorical_features",
                    FieldName.FEAT_STATIC_REAL: "static_real_features",
                    FieldName.FEAT_TIME: "time_features",
                    FieldName.TARGET: "values",
                    FieldName.OBSERVED_VALUES: "observed_mask",
                }
            ),
        ]
    )


In [ ]:
from gluonts.transform.sampler import InstanceSampler
from typing import Optional


def create_instance_splitter(config: PretrainedConfig, mode: str, train_sampler: Optional[InstanceSampler] = None,
                             validation_sampler: Optional[InstanceSampler] = None, ) -> Transformation:
    assert mode in ["train", "validation", "test"]

    instance_sampler = {
        "train": train_sampler or ExpectedNumInstanceSampler(
            num_instances=1.0, min_future=config.prediction_length
        ),
        "validation": validation_sampler or ValidationSplitSampler(
            min_future=config.prediction_length
        ),
        "test": TestSplitSampler(),
    }[mode]

    return InstanceSplitter(
        target_field="values",
        is_pad_field=FieldName.IS_PAD,
        start_field=FieldName.START,
        forecast_start_field=FieldName.FORECAST_START,
        instance_sampler=instance_sampler,
        past_length=config.context_length + max(config.lags_sequence),
        future_length=config.prediction_length,
        time_series_fields=[
            "time_features",
            "observed_mask",
        ],
    )



In [ ]:
from gluonts.itertools import Cyclic, IterableSlice, PseudoShuffled
from gluonts.torch.util import IterableDataset
from torch.utils.data import DataLoader

from typing import Iterable


def create_train_dataloader(
        config: PretrainedConfig,
        freq,
        data,
        batch_size: int,
        num_batches_per_epoch: int,
        shuffle_buffer_length: Optional[int] = None,
        **kwargs,
) -> Iterable:
    PREDICTION_INPUT_NAMES = [
        "static_categorical_features",
        "static_real_features",
        "past_time_features",
        "past_values",
        "past_observed_mask",
        "future_time_features",
    ]

    TRAINING_INPUT_NAMES = PREDICTION_INPUT_NAMES + [
        "future_values",
        "future_observed_mask",
    ]

    transformation = create_transformation(freq, config)
    transformed_data = transformation.apply(data, is_train=True)

    # we initialize a Training instance
    instance_splitter = create_instance_splitter(
        config, "train"
    ) + SelectFields(TRAINING_INPUT_NAMES)

    # the instance splitter will sample a window of
    # context length + lags + prediction length (from the 366 possible transformed time series)
    # randomly from within the target time series and return an iterator.
    training_instances = instance_splitter.apply(
        Cyclic(transformed_data)
        if shuffle_buffer_length is None
        else PseudoShuffled(
            Cyclic(transformed_data),
            shuffle_buffer_length=shuffle_buffer_length,
        )
    )

    # from the training instances iterator we now return a Dataloader which will
    # continue to sample random windows for as long as it is called
    # to return batch_size of the appropriate tensors ready for training!
    return IterableSlice(
        iter(
            DataLoader(
                IterableDataset(training_instances),
                batch_size=batch_size,
                **kwargs,
            )
        ),
        num_batches_per_epoch,
    )



In [ ]:
train_dataloader = create_train_dataloader(
    config=config,
    freq=freq,
    data=train_dataset,
    batch_size=16,
    num_batches_per_epoch=128,
)

batch = next(iter(train_dataloader))

for k, v in batch.items():
    print("{: >30} {: <30} {: >10}".format(k, str(v.shape), v.type()))


In [ ]:
def create_test_dataloader(
        config: PretrainedConfig,
        freq,
        data,
        batch_size: int,
        **kwargs,
):
    PREDICTION_INPUT_NAMES = [
        "static_categorical_features",
        "static_real_features",
        "past_time_features",
        "past_values",
        "past_observed_mask",
        "future_time_features",
    ]

    new_data = [i for i, _ in data]

    transformation = create_transformation(freq, config)
    transformed_data = transformation.apply(new_data, is_train=False)

    # we create a Test Instance splitter which will sample the very last
    # context window seen during training only for the encoder.
    instance_sampler = create_instance_splitter(
        config, "validation"
    ) + SelectFields(PREDICTION_INPUT_NAMES)

    # we apply the transformations in test mode
    testing_instances = instance_sampler.apply(transformed_data, is_train=False)

    # This returns a Dataloader which will go over the dataset once.
    return DataLoader(IterableDataset(testing_instances), batch_size=batch_size, **kwargs)



In [ ]:
train_dataloader = create_train_dataloader(
    config=config,
    freq=freq,
    data=train_dataset,
    batch_size=16,
    num_batches_per_epoch=100,
)

# TODO test dataset has only one data now.
test_dataloader = create_test_dataloader(
    config=config,
    freq=freq,
    data=test_pairs,
    batch_size=64,
)


In [ ]:
batch = next(iter(test_dataloader))

for k, v in batch.items():
    print("{: >30} {: <30} {: >10}".format(k, str(v.shape), v.type()))

batch


In [ ]:
batch = next(iter(train_dataloader))

for k, v in batch.items():
    print("{: >30} {: <30} {: >10}".format(k, str(v.shape), v.type()))


In [ ]:
outputs = model(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_values=batch["future_values"],
    future_time_features=batch["future_time_features"],
    future_observed_mask=batch["future_observed_mask"],
    output_hidden_states=True
)

outputs


In [ ]:
print("Loss:", outputs.loss.item())


In [ ]:
from accelerate import Accelerator
from torch.optim import Adam
from tqdm import tqdm

accelerator = Accelerator()
device = accelerator.device

model.to(device)
optimizer = Adam(model.parameters(), lr=1e-4)

model, optimizer, train_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader,
)

epoch_num = 1000

model.train()
for epoch in tqdm(range(epoch_num)):
    for batch in train_dataloader:
        optimizer.zero_grad()
        outputs = model(
            static_categorical_features=batch["static_categorical_features"].to(device),
            static_real_features=batch["static_real_features"].to(device),
            past_time_features=batch["past_time_features"].to(device),
            past_values=batch["past_values"].to(device),
            future_time_features=batch["future_time_features"].to(device),
            future_values=batch["future_values"].to(device),
            past_observed_mask=batch["past_observed_mask"].to(device),
            future_observed_mask=batch["future_observed_mask"].to(device),
        )
        loss = outputs.loss

        # Backpropagation
        accelerator.backward(loss)
        optimizer.step()

        # print(loss.item())



  1%|          | 8/1000 [00:22<46:18,  2.80s/it]


KeyboardInterrupt: 

In [ ]:
import torch

torch.save(model.state_dict(), 'weight/test.pt')


In [ ]:
model.eval()

forecasts = []

for batch in test_dataloader:
    outputs = model.generate(
        static_categorical_features=batch["static_categorical_features"].to(device),
        static_real_features=batch["static_real_features"].to(device),
        past_time_features=batch["past_time_features"].to(device),
        past_values=batch["past_values"].to(device),
        future_time_features=batch["future_time_features"].to(device),
        past_observed_mask=batch["past_observed_mask"].to(device),
    )
    forecasts.append(outputs.sequences.cpu().numpy())

forecasts = np.vstack(forecasts)
forecasts.shape


In [ ]:
forecast_median = np.median(forecasts, 1)
forecast_median_standardized = (forecast_median - dataset_mean) / dataset_std
forecast_median_standardized = forecast_median_standardized[:, 0]

# np.median(forecasts[0][0])
# forecast_median.shape
forecast_median_standardized.shape

# forecasts.shape
# forecasts[:, 0, :, 0]
# np.median(forecasts[:, 0])


In [ ]:
from evaluate import load

mase_metric = load("evaluate-metric/mase")
smape_metric = load("evaluate-metric/smape")

mae_metric = load("evaluate-metric/mae")
mse_metric = load("evaluate-metric/mse")
mape_metric = load("evaluate-metric/mape")


In [ ]:
ground_truth = [test_label['target'][0] for test_input, test_label in test_pairs]
ground_truth = (ground_truth - dataset_mean) / dataset_std
ground_truth = np.array(ground_truth)

is_finite = np.isfinite(ground_truth)

predictions = forecast_median_standardized[is_finite]
references = ground_truth[is_finite]

mae = mae_metric.compute(
    predictions=predictions,
    references=references,
)
mae_metrics = mae["mae"]

mse = mse_metric.compute(
    predictions=predictions,
    references=references,
    squared=False,
)
mse_metrics = mse["mse"]

mape = mape_metric.compute(
    predictions=predictions,
    references=references,
)
mape_metrics = mape["mape"]


In [ ]:
print(f"MAE: {mae_metrics}")
print(f"RMSE: {mse_metrics}")
print(f"MAPE: {mape_metrics}")


In [ ]:
import matplotlib.dates as mdates

fig, ax = plt.subplots()

display_size = split_size * 2
actual_value = df['Close'][-display_size:]
# actual_value = test_dataset[FieldName.TARGET][-1][-display_size:]

df_actual_value = pd.DataFrame(actual_value)
df_actual_value = df_actual_value.fillna(method='ffill')

index = pd.period_range(
    # start=test_pairs[FieldName.START][0],
    start=split_date,
    periods=len(actual_value),
    freq=freq,
).to_timestamp()

# Major ticks every half year, minor ticks every month,
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 13)))
ax.xaxis.set_minor_locator(mdates.MonthLocator())

ax.plot(
    index,
    df_actual_value,
    label="actual",
)

plt.plot(
    index[-split_size + 1:],
    np.median(forecasts, axis=1),
    label="median",
)

forecasts_mean = np.mean(forecasts, axis=1)[:, 0]
forecasts_std = np.std(forecasts, axis=1)[:, 0]

plt.fill_between(
    index[-split_size + 1:],
    forecasts_mean - forecasts_std,
    forecasts_mean + forecasts_std,
    alpha=0.3,
    interpolate=True,
    label="+/- 1-std",
)

plt.legend()
plt.show()



In [ ]:

# Major ticks every half year, minor ticks every month,
# ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))
# ax.xaxis.set_minor_locator(mdates.MonthLocator())
#
# ax.plot(
#     index[-2 * prediction_length:],
#     test_dataset[ts_index]["target"][-2 * prediction_length:],
#     label="actual",
# )
#
# plt.plot(
#     index[-prediction_length:],
#     np.median(forecasts[ts_index], axis=0),
#     label="median",
# )
#
# plt.fill_between(
#     index[-prediction_length:],
#     forecasts[ts_index].mean(0) - forecasts[ts_index].std(axis=0),
#     forecasts[ts_index].mean(0) + forecasts[ts_index].std(axis=0),
#     alpha=0.3,
#     interpolate=True,
#     label="+/- 1-std",
# )
# plt.legend()
# plt.show()
